In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
dataDirectory = "imgs/" # folder that contains the dataset

In [2]:
img_size = 224 #imagenetv2 input size
classes = ["0", "1", "2", "3", "4", "5", "6"] #list of classes
trainingData = []

In [4]:
def create_training_data():
    for category in classes:
        img_path = os.path.join(dataDirectory, category)
        for img in os.listdir(img_path):
            try:
                img_array = cv2.imread(os.path.join(img_path, img))
                resized_img = cv2.resize(img_array, (img_size, img_size))
                trainingData.append([resized_img, int(category)])
            except Exception as e :
                pass

In [5]:
create_training_data()

In [6]:
import random
random.shuffle(trainingData)

In [7]:
X, y = [], []

for img, label in trainingData:
    X.append(img)
    y.append(label)

In [8]:
X = np.array(X).reshape(-1, img_size, img_size, 3)
X = X / 255.0 #normalize the data

y = np.array(y)

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
model = tf.keras.applications.MobileNetV2()

In [12]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [13]:
input = model.layers[0].input
output = model.layers[-2].output

In [14]:
finalOutput = layers.Dense(128)(output) # adding dense layer after GlobalAveragePooling layer
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(64)(finalOutput_)
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(7, activation='softmax')(finalOutput_) # we are working with 7 classes

In [16]:
model = keras.Model(inputs= input, outputs= finalOutput)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [18]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X, y, epochs=20)

417/417 [==============================] - 484s 1s/step - loss: 1.4118 - accuracy: 0.4690


In [ ]:
model.save("final_model.h5")